In [1]:
!pip install scipy openai tiktoken retry dashscope loguru

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 9.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 18.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


并行处理相关包
from multiprocessing import Process, Manager
Process：用于创建和管理独立的进程，支持并行执行任务。
Manager：提供了一种在进程之间共享数据的方式，如共享列表、字典等。
用途：在多核处理器上并行执行多个任务，提高程序性能。

数据处理与显示相关包
import json
用途：用于解析和生成JSON格式的数据。
import os
用途：提供与操作系统交互的功能，如文件和目录操作。
from pprint import pprint
用途：以更具可读性的方式打印数据结构，尤其适合打印嵌套的字典或列表。
import re
用途：用于正则表达式操作，如模式匹配和字符串搜索。
from tqdm import tqdm
用途：为循环添加进度条，便于跟踪任务的进度。
import random
用途：生成随机数或进行随机选择，用于模拟和随机化操作。
通用工具和库
import uuid
用途：生成唯一标识符（UUID），用于标识唯一的对象或请求。
import numpy as np
用途：用于数值计算和数组操作，提供了高效的多维数组对象。
import requests
用途：用于发送HTTP请求，进行网络通信。
from retry import retry
用途：实现自动重试机制，处理偶发性错误或失败。
科学计算与稀疏矩阵
from scipy import sparse
用途：用于处理和操作稀疏矩阵，适用于大规模稀疏数据集。
并行执行相关包
from concurrent.futures import ThreadPoolExecutor, as_completed
ThreadPoolExecutor：用于管理和执行线程池，支持并行执行多个任务。
as_completed：用于迭代已完成的任务，按完成顺序返回结果。
用途：并行执行多个任务，提高程序执行效率。

日志记录相关包
from loguru import logger
用途：用于日志记录，提供简洁且功能强大的日志记录API。
时间处理相关包
import time
用途：提供时间相关功能，如获取当前时间、休眠等。
具体领域相关库
import openai
用途：用于与OpenAI的API进行交互，调用GPT模型等。
import tiktoken
用途：用于处理和解析文本中的token，常用于自然语言处理任务。
import dashscope
用途：用于与Dashscope平台进行通信，调用模型推理服务。

In [2]:
from multiprocessing import Process, Manager
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
import tiktoken
import json
import numpy as np
import requests
from retry import retry
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus
import dashscope


from concurrent.futures import ThreadPoolExecutor, as_completed
from loguru import logger
import json
import time
from tqdm import tqdm


In [ ]:
logger.remove()  # 移除默认的控制台输出
logger.add("logs/app_{time:YYYY-MM-DD}.log", level="INFO", rotation="00:00", retention="10 days", compression="zip")

MODEL_NAME = 'qwen1.5-1.8b-chat'  # 截止到7.26日下午15:00，该模型在灵积平台限时免费

### 1. logger.remove()
功能：这行代码用于移除loguru日志记录器的默认控制台输出。loguru库在默认情况下会将日志输出到控制台（即终端或命令行窗口）。使用logger.remove()可以移除这种默认行为，以便自定义日志输出配置。
### 2. logger.add("logs/app_{time:YYYY-MM-DD}.log", level="INFO", rotation="00:00", retention="10 days", compression="zip")
功能：这行代码用于添加一个新的日志记录器，并将日志记录到指定的文件中，同时设定日志文件的旋转、保留和压缩策略。
参数：
- "logs/app_{time:YYYY-MM-DD}.log"：日志文件的路径和名称，其中{time:YYYY-MM-DD}表示日志文件名将包含日志创建的日期。例如，2024年7月28日的日志文件名将是app_2024-07-28.log。
- level="INFO"：设定日志记录的最低级别为INFO。这意味着INFO级别及以上的日志（如WARNING、ERROR、CRITICAL）将被记录，低于INFO级别的日志（如DEBUG）将被忽略。
- rotation="00:00"：设定日志文件每天午夜进行旋转（创建新日志文件）。这意味着每天将生成一个新的日志文件。
- retention="10 days"：设定日志文件的保留期限为10天。超过10天的日志文件将被自动删除。
- compression="zip"：设定日志文件的压缩方式为zip。旋转后的旧日志文件将被压缩为zip格式以节省存储空间。
### 3. MODEL_NAME = 'qwen1.5-1.8b-chat'
功能：这行代码定义了一个变量MODEL_NAME，并将其值设定为模型的名称 'qwen1.5-1.8b-chat'。
用途：该变量通常用于在代码中引用特定的机器学习模型或自然语言处理模型的名称。在实际应用中，可能会用到该变量来加载或调用该模型。例如，MODEL_NAME可能在调用API或模型推理过程中使用，以便指定要使用的模型版本。

In [3]:
# 注意：这里需要填入你的key~ 咱们在第二步申请的。
dashscope.api_key=""

这段代码定义了一个名为 call_qwen_api 的函数，该函数用于调用一个名为 dashscope.Generation 的 API 来生成文本。以下是该代码的功能、用途和特点的详细介绍：

* 功能
调用 API 生成文本：该函数通过传递一个模型名称 (MODEL_NAME) 和一个查询 (query) 来调用 dashscope.Generation.call 方法，生成相应的文本。
处理 API 响应：函数会检查 API 的响应状态码，如果状态码为 HTTPStatus.OK，则提取并返回生成的文本内容。如果状态码不是 HTTPStatus.OK，则打印错误信息并抛出异常。
* 用途
文本生成：该函数主要用于通过调用外部 API 来生成文本，适用于需要动态生成内容的场景，如聊天机器人、内容创作辅助等。
错误处理：通过检查 API 响应状态码并处理错误情况，确保在调用失败时能够及时发现并处理问题。
* 特点
重试机制：函数使用了 @retry(delay=3, tries=3) 装饰器，这意味着在调用 API 失败时，函数会自动重试最多 3 次，每次重试间隔 3 秒。
消息格式：在调用 API 时，设置了 result_format='message'，表示期望的响应格式是消息格式。
错误处理：在 API 调用失败时，函数会打印详细的错误信息，包括请求 ID、状态码、错误代码和错误消息，并抛出异常，以便上层调用者能够捕获并处理这些错误。

In [4]:
# 函数 api_retry 的作用是实现对一个API调用的重试机制，以应对可能的失败情况。
# 它尝试调用 call_qwen_api 的函数，如果调用失败，会按照设定的重试策略重新尝试，直到成功或者达到最大重试次数。
def api_retry(MODEL_NAME, query):
    #MODEL_NAME: string，表示要使用的模型名称。
    #query: string，表示要发送到API的查询文本。

    max_retries = 5   #最大重试次数，设定为5次。如果调用失败，会最多尝试5次。
    retry_delay = 60  #每次重试之间的延迟时间，设定为60秒。
    attempts = 0      #当前尝试的次数，初始值为0。
    
    while attempts < max_retries:
        try:
            return call_qwen_api(MODEL_NAME, query)
        except Exception as e:
            attempts += 1   
            if attempts < max_retries:
                logger.warning(f"Attempt {attempts} failed for text: {query}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                logger.error(f"All {max_retries} attempts failed for text: {query}. Error: {e}")
                raise

初始化和循环开始:

初始化 attempts 变量为0。
使用 while 循环，循环条件是 attempts < max_retries，即最多重试 max_retries 次。
尝试调用API:

在 try 代码块中，调用 call_qwen_api(MODEL_NAME, query) 函数。
如果调用成功，则返回结果，函数执行结束。
异常处理:

如果调用失败，抛出异常，进入 except 块。
将 attempts 计数器加1。
重试逻辑:

检查 attempts 是否小于 max_retries：
如果是，则记录一个警告日志，表示第几次尝试失败，并将在 retry_delay 秒后重试。
使用 time.sleep(retry_delay) 使程序等待 retry_delay 秒，然后继续下一次尝试。
如果 attempts 达到或超过 max_retries：
记录一个错误日志，表示所有尝试均已失败，并包括异常信息 e。
使用 raise 重新抛出异常，使调用此函数的代码可以捕获并处理该异常。


except Exception as e: 是异常处理机制的一部分。它用于捕获并处理在 try 代码块中发生的所有异常。
详细解释
异常处理基础：

try 块：包含可能引发异常的代码。
except 块：处理在 try 块中引发的异常。
Exception 类：

Exception 是Python内置的所有异常的基类。捕获 Exception 可以捕获几乎所有的异常，但不包括系统退出事件（如 SystemExit、 KeyboardInterrupt 等）。
这是一个广义的捕获，适用于大多数普通的异常情况。
as e 部分：

as e 将捕获的异常实例赋值给变量 e，便于在 except 块中访问异常对象。
通过 e 可以获取异常的详细信息，如异常消息、异常类型等。

In [5]:
# call_qwen_api 的主要目的是使用 Dashscope API 调用指定的模型进行推理，并通过 HTTP 传输 JSON 格式的数据来返回结果。
def call_qwen_api(MODEL_NAME, query):
    # 1.构建消息
    # 创建一个包含用户查询的消息列表。
    # 消息格式符合对话模型的输入要求，其中 role 为 'user'，content 为用户的查询内容。
    messages = [
        {'role': 'user', 'content': query}]
    # 2.调用 Dashscope API
    # 使用 Dashscope 提供的 Generation.call 方法调用模型推理。
    # 传递的参数包括模型名称 MODEL_NAME、消息列表 messages，以及结果格式 result_format 设置为 'message'。
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    #3.处理响应
    # 检查响应的状态码是否为 HTTP 200（OK）。
    # 如果是（表示 HTTP 200 状态码，表示请求成功），则从响应中提取并返回模型生成的文本内容。
    # 如果不是，则打印请求ID、状态码、错误码和错误信息，并抛出异常
    if response.status_code == HTTPStatus.OK:
        # print(response)
        return response['output']['choices'][0]['message']['content']#这里的详解在下面：
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

response['output']: 访问响应字典中的 output 键。output 键的值是一个包含进一步信息的字典。
response['output']['choices']: 访问 output 字典中的 choices 键。choices 键的值是一个列表，列表中的每个元素通常表示一个可能的回答或选项。
response['output']['choices'][0]: 从 choices 列表中选择第一个元素（索引为0的元素）。在大多数API响应中，choices 列表包含多个可能的结果，而我们通常只需要第一个（最相关的）结果。
response['output']['choices'][0]['message']: 访问第一个 choices 元素中的 message 键。message 键的值是一个包含消息内容的字典。
response['output']['choices'][0]['message']['content']: 访问 message 字典中的 content 键。content 键的值是实际的文本内容，即模型生成的响应。

In [6]:
# 这里定义了prompt推理模版
# get_prompt 用于生成一个用于逻辑推理的提示模板。
# 通过将问题、问题描述和选项组合成一个结构化的文本，便于逻辑推理模型处理并生成答案。

def get_prompt(problem, question, options):
# 1.函数定义和参数:
# problem：表示逻辑推理题目的整体描述。
# question：表示具体的问题。
# options：表示可能的选项，是一个列表。

# 2.生成选项文本
# 使用列表推导和 join 方法，将选项列表转换为格式化的字符串，每个选项占一行，前缀为选项字母（A, B, C, D, E, F, G）。
# enumerate(options) 会为选项提供索引，'ABCDEFG'[i] 会根据索引生成相应的字母前缀。
    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))
# 3.生成提示文本
    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""
    # 4. 返回生成的提示:
    # print(prompt)
    return prompt

#- 使用三重引号定义多行字符串模板。
#- 在模板中插入 `problem`、`question` 和 `options` 的内容，将它们格式化为完整的提示文本。
#- 这段文本假设了一个逻辑推理专家，并详细说明了题目的类型和假设。


In [7]:
# 这里使用extract抽取模获得抽取的结果
# 函数 extract 的作用是从给定的文本中提取答案。
# 假设答案在文本中按照特定的格式出现，即 "答案是：X" 的形式，其中 X 是一个选项（如 A、B、C 等）。

def extract(input_text):
    # 1.定义正则表达式模板
    # re.compile 用于编译正则表达式模式，使得后续匹配操作更高效。
    # 模式 r"答案是：(.)"：
    # 答案是： 匹配固定的文本。
    # (.) 是一个捕获组，匹配任何单个字符（通常是答案选项如 A、B、C 等）。
    # re.S 是一个标志，表示点号 . 也可以匹配换行符。
    ans_pattern = re.compile(r"答案是：(.)", re.S)

    # 2.查找匹配项
    # findall 方法返回一个列表，包含所有与正则表达式模式匹配的内容。
    # 如果 input_text 中有多个 "答案是：X" 的出现，findall 会返回所有匹配的结果组成的列表。
    problems = ans_pattern.findall(input_text)
    # print(problems)
    # 3.检查并返回结果
    # if problems == '' 检查 problems 列表是否为空。
    # 如果 problems 为空，意味着没有找到匹配项。
    # 返回 'A' 作为默认值。
    # 如果找到匹配项，则返回第一个匹配的答案 problems[0]——最优解。
    if(problems == ''):
        return 'A'
    return problems[0]

In [8]:
# 函数 process_datas 的主要作用是并行处理多个数据项，通过生成提示文本、调用API获取结果、并提取和保存答案。
# 1.def函数 process_datas，接受两个参数 datas 和 MODEL_NAME。
# 初始化空列表 results 用于存储处理后的结果。
def process_datas(datas,MODEL_NAME):
    results = []

# 2.创建线程池执行器
# 使用 ThreadPoolExecutor 创建一个最大并发线程数为16的线程池。
# 初始化 future_data 字典用于存储任务与数据的映射关系，lasttask 和 lastmark 初始化用于后续处理，lens 初始化为0，用于记录总任务数。
    with ThreadPoolExecutor(max_workers=16) as executor:
        future_data = {}
        lasttask = ''
        lastmark = 0
        lens = 0

    # 3.提交任务
    # 使用 tqdm 显示任务提交进度。
    # 对每个 data，提取 problem 和 questions。
    # 使用 get_prompt 生成提示文本 prompt。
    # 提交 api_retry 任务到线程池，并将 future 和对应的数据项 data 及问题 id 存入 future_data 字典。
    # 通过 time.sleep(0.6) 控制任务提交频率，防止API过载。
        for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):
            problem = data['problem']
            for id,question in enumerate(data['questions']):
                prompt = get_prompt(problem, 
                                    question['question'], 
                                    question['options'],
                                    )

                future = executor.submit(api_retry, MODEL_NAME, prompt)
                
                future_data[future] = (data,id)
                time.sleep(0.6)  # 控制每0.5秒提交一个任务
                lens += 1

        # 4，处理任务结果
        # 使用 tqdm 显示任务处理进度。
        # 对每个已完成的任务，从 future_data 中获取对应的 data 和 problem_id。
        # 调用 future.result() 获取任务结果 res，并使用 extract 提取答案 extract_response。
        # 将提取的答案保存到 data['questions'][problem_id]['answer']，并将更新后的 data 添加到 results 列表中。
        # 如果任务执行失败，记录错误日志。
        for future in tqdm(as_completed(future_data), total=lens, desc="Processing tasks"):
            # print('data',data)
            data = future_data[future][0]
            problem_id = future_data[future][1]
            try:
                res  = future.result()
                extract_response = extract(res)
                # print('res',extract_response)
                data['questions'][problem_id]['answer'] = extract_response
                results.append(data)
                # print('data',data)
                
            except Exception as e:
                logger.error(f"Failed to process text: {data}. Error: {e}")
    
    # 5.返回包含所有处理后数据的 results 列表。
    return results

In [9]:
# 函数 main 的作用是读取输入文件中的数据、处理这些数据，然后将结果返回。
def main(ifn, ofn):
    # 1.检查输出文件是否存在
    # if os.path.exists(ofn): 检查路径 ofn 指定的文件是否存在。
    # pass: 如果文件存在，则什么都不做。这里的逻辑没有具体操作，可能是为了将来添加功能或为了与现有代码结构一致。
    if os.path.exists(ofn):
        pass

    # 2.初始化数据列表
    # 初始化一个空列表 data，用于存储从输入文件中读取的数据。
    data = []

    # 3.按行读取数据
    # with open(ifn) as reader: 以只读模式打开文件 ifn，并将文件对象赋值给 reader。
    # for line in reader: 逐行读取文件内容。
    # sample = json.loads(line): 使用 json.loads 将每一行的 JSON 格式字符串解析为 Python 字典或列表。
    # data.append(sample): 将解析后的数据追加到 data 列表中。
    with open(ifn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

    # 4.复制数据列表
    # 将读取并解析后的数据列表 data 赋值给变量 datas。
    # 这里 datas 是 data 的一个引用。
    datas = data
    # print(data)
    # 5.处理数据
    # 调用 process_datas 函数，传入 datas 和 MODEL_NAME，处理数据并返回处理结果列表 return_list。
    # 均匀地分成多个数据集
    return_list = process_datas(datas,MODEL_NAME)

    # 6.打印结果数量和任务完成信息
    # print(len(return_list)): 打印处理后的结果列表的长度，即处理完成的数据项数量。
    # print("All tasks finished!"): 打印任务完成的信息。
    print(len(return_list))
    print("All tasks finished!")

    # 7.返回处理后的结果列表 return_list。
    return return_list

In [10]:
# 函数 evaluate 的作用是读取输出文件中的数据，并根据特定的评价指标对数据进行评估。
# 具体来说，它会计算模型预测的准确性。
def evaluate(ofn):
    # 1.初始化数据列表
    # 初始化一个空列表 data，用于存储从输出文件中读取的数据。
    data = []

    # 2.按行读取数据
    # with open(ofn) as reader: 以只读模式打开文件 ofn，并将文件对象赋值给 reader。
    # for line in reader: 逐行读取文件内容。
    # sample = json.loads(line): 使用 json.loads 将每一行的 JSON 格式字符串解析为 Python 字典或列表。
    # data.append(sample): 将解析后的数据追加到 data 列表中。
    with open(ofn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

    # 3.初始化计数器
    # pse: 记录没有模型预测答案的题目数量。
    # cnt: 记录模型预测正确的题目数量。
    # tot: 记录总的题目数量（有模型预测答案的）。
    pse = 0
    cnt = 0
    tot = 0

    # 4.遍历数据并计算准确性
    # for task in data: 遍历每一个任务（每个任务包含一个问题集）。
    # for question in task['questions']: 遍历任务中的每个问题。
    # if MODEL_NAME in question: 检查当前问题是否包含模型的预测结果。
    # 如果包含，则:
    # tot += 1: 增加总题目数量计数器。
    # cnt += question[MODEL_NAME] == question['answer']: 如果模型的预测结果与正确答案相同，增加正确答案计数器。
    # 如果不包含:
    # pse += 1: 增加没有模型预测答案的题目数量计数器。
    for task in data:
        for question in task['questions']:
            
            if MODEL_NAME in question:
                tot += 1
                cnt += question[MODEL_NAME] == question['answer']
            else:
                pse += 1
    # 5.打印计数器结果：
    # cnt: 模型预测正确的题目数量。
    # tot: 总的有模型预测答案的题目数量。
    # cnt/tot: 模型预测的准确率（正确预测数量占总预测数量的比例）。
    # pse: 没有模型预测答案的题目数量。
    print(cnt, tot, cnt/tot, pse)

In [11]:
# 定义了 __main__ 的运行入口，并执行了一些操作，包括调用 extract 函数和 main 函数。
# 经典if __name__ == '__main__':
# 这一行代码是一个常见的Python习惯用法，用于确保代码块只在模块作为主程序运行时执行，而在被导入其他模块时不执行。
if __name__ == '__main__':

# 调用extract_function

# 1.调用 extract 函数:
# 传入一个包含算法步骤和结论的长字符串。
# 目标是从文本中提取出最后的答案（根据格式"答案是：X"），即"C"。

# 2.解释 extract 函数的执行:
# 使用正则表达式匹配文本中 "答案是：" 后的字符。
# 返回匹配到的字符。

# 3.打印提取的答案，输出结果应该是"C"。

    a = extract("""根据欧几里得算法，逐步解析计算两个数6和7的最大公约数（gcd）的步骤如下：

1. 判断6和7是否相等：不相等。
2. 判断6和7大小关系，7 > 6，所以用更大的数7减去较小的数6得到结果1。
3. 现在计算6和1的最大公约数。
4. 6 > 1，根据算法用更大的数6减去较小的数1得到结果5。
5. 再计算5和1的最大公约数。
6. 5 > 1，用5减去1得到结果4。
7. 再计算4和1的最大公约数。
8. 4 > 1，用4减去1得到结果3。
9. 再计算3和1的最大公约数。
10. 3 > 1，用3减去1得到结果2。
11. 再计算2和1的最大公约数。
12. 2 > 1，用2减去1得到结果1。
13. 最后计算1和1的最大公约数，两数相等，gcd即为这两个数，也就是1。

因此，6和7的最大公约数是1。

答案是：C.""")

    print(a)

    # main()
    # 1.传入2个文件路径
    # 'round1_test_data.jsonl'：输入文件路径，包含逻辑推理题目的JSON行文件。
    # 'upload.jsonl'：输出文件路径，处理结果将保存到这个文件。
    return_list = main('round1_test_data.jsonl', 'upload.jsonl')


C


Submitting tasks:  52%|█████▏    | 262/500 [06:56<06:25,  1.62s/it]

Request id: 4f83e33b-3dc0-932f-bdd0-b4553b2c10a8, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


Submitting tasks:  59%|█████▉    | 297/500 [07:56<06:58,  2.06s/it]

Request id: 10183a8e-ceba-9637-bfab-56ddd3d54ab4, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: 39a902cd-aaee-9240-ba3c-ee147bf8ee81, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: cb7bea8e-10e7-95d9-9088-be44b415b63d, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


...

Request id: b87584da-83e7-9c3f-b217-1c6c29bb4637, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


Processing tasks:  98%|█████████▊| 1296/1328 [09:06<00:23,  1.36it/s]

Request id: 7e649e98-329f-9967-b6c8-9dc0d17bede3, Status code: 500, error code: RequestTimeOut, error message: Request timed out, please try again later.


Processing tasks: 100%|██████████| 1328/1328 [11:05<00:00,  1.99it/s]

764
All tasks finished!


#### 调用 `main` 函数

```python
return_list = main('round1_test_data.jsonl', 'upload.jsonl')
```

1. **传入两个文件路径**:
   - `'round1_test_data.jsonl'`：输入文件路径，包含逻辑推理题目的JSON行文件。
   - `'upload.jsonl'`：输出文件路径，处理结果将保存到这个文件。

2. **`main` 函数的详细解释**:

   ```python
   def main(ifn, ofn):
       if os.path.exists(ofn):
           pass
       data = []
       with open(ifn) as reader:
           for line in reader:
               sample = json.loads(line)
               data.append(sample)
       datas = data
       return_list = process_datas(datas, MODEL_NAME)
       print(len(return_list))
       print("All tasks finished!")
       return return_list
   ```

   - **检查输出文件是否存在**：
     ```python
     if os.path.exists(ofn):
         pass
     ```
     - 如果输出文件存在，当前没有任何处理（可能是为了将来扩展）。

   - **读取输入文件内容**：
     ```python
     with open(ifn) as reader:
         for line in reader:
             sample = json.loads(line)
             data.append(sample)
     ```
     - 逐行读取输入文件，将每行的JSON格式数据解析为Python对象，并存储在列表 `data` 中。

   - **处理数据**：
     ```python
     datas = data
     return_list = process_datas(datas, MODEL_NAME)
     ```
     - 将读取的数据传递给 `process_datas` 函数进行处理。
     - `MODEL_NAME` 应该是在模块其他地方定义的模型名称，用于调用API进行推理。

   - **输出处理结果数量和任务完成信息**：
     ```python
     print(len(return_list))
     print("All tasks finished!")
     ```
     - 打印处理后的数据项数量。
     - 打印任务完成的信息。

   - **返回处理后的结果列表**：
     ```python
     return return_list
     ```

### 代码执行顺序和结果

1. **提取答案**:
   - 调用 `extract` 函数并打印结果，确认从文本中提取的答案为 "C"。

2. **处理逻辑推理题目数据**:
   - 调用 `main` 函数，读取输入文件 `round1_test_data.jsonl` 中的逻辑推理题目。
   - 使用 `process_datas` 函数并行处理这些数据，调用API获取答案并保存结果。
   - 打印处理后的数据项数量和任务完成信息。
   - 返回处理后的结果列表。

In [12]:
# 函数 has_complete_answer 作用是检查问题列表是否包含完整答案
# 1.def 函数
# 定义函数 has_complete_answer，接受一个参数 questions，表示问题列表。
def has_complete_answer(questions):
    # 2.判断逻辑
    # 这里假设完整答案的判断逻辑是：每个question都有一个'answer'键
    # 遍历问题列表:
    # 遍历 questions 列表中的每一个问题 question。
    for question in questions:
    # 检查是否有 answer 键:
    # 如果某个 question 中没有 answer 键，返回 False，表示该问题列表不包含完整答案。
        if 'answer' not in question:
            return False
        
    # return True   
    # 如果所有问题都包含 answer 键，返回 True，表示问题列表包含完整答案。
    return True


# 函数 filter_problems 作用是根据has_complete_answer这个条件——过滤和处理数据列表。
# 1.def filter_problems
# 定义函数 filter_problems，接受一个参数 data，表示包含多个问题集合的数据列表。
def filter_problems(data):
    
    # 2.初始化
    # 初始化一个空列表 result，用于存储筛选后的结果。
    result = []
    # 初始化一个空集合 problem_set，用于跟踪已经处理过的问题。
    problem_set = set()
    
    # 遍历 data 列表中的每一个项目 item，提取问题描述 problem。
    for item in data:
        problem = item['problem']
        # print('处理的item' ,item)

        # 4.检查问题是否已经处理
        # 如果 problem 已存在于 problem_set 中，表示这个问题之前已经处理过。
        # 遍历 result 列表，找到对应的 existing_item。
        # 如果当前 item 中的问题列表 questions 包含完整答案（使用 has_complete_answer 函数检查），则更新 existing_item 中的问题列表和ID。
        if problem in problem_set:
            # 找到已存在的字典
            for existing_item in result:
                if existing_item['problem'] == problem:
                    # 如果当前字典有完整答案，替换已存在的字典
                    if has_complete_answer(item['questions']):
                        existing_item['questions'] = item['questions']
                        existing_item['id'] = item['id']
                    break
        
        # 5.检查新问题
        # 如果 problem 不在 problem_set 中，表示这是一个新问题。
        # 如果当前 item 中的问题列表 questions 包含完整答案，则将 item 添加到 result 列表，并将 problem 添加到 problem_set 中。
        else:
            # 如果当前字典有完整答案，添加到结果列表
            if has_complete_answer(item['questions']):
                result.append(item)
                problem_set.add(problem)

    return result

In [13]:
# 对 return_list 进行进一步处理，包括过滤问题、排序数据并打印排序结果。
return_list
# 调用 filter_problems 函数，对 return_list 中的数据进行过滤，保留包含完整答案的问题。
return_list = filter_problems(return_list)

# 使用 sorted 函数对 return_list 进行排序，按照 id 的最后三位数字进行排序。
sorted_data = sorted(return_list, key=lambda x: int(str(x['id'])[-3:]))
# 排序逻辑如下：
# key=lambda x: int(str(x['id'])[-3:]):
# x['id']：获取数据项 x 的 id 值。
# str(x['id'])：将 id 转换为字符串。
# str(x['id'])[-3:]：获取 id 字符串的最后三位。
# int(str(x['id'])[-3:])：将最后三位字符串转换为整数，作为排序键。
print(sorted_data)

[{'problem': '有一个列表，找出该列表的最后一个元素。\n\n下列选项中哪个是列表 `[a, b, c, d]` 的最后一个元素？', 'questions': [{'question': '选择题 1：', 'options': ['a', 'b', 'c', 'd'], 'answer': '不'}], 'id': 'round1_test_data_002'}, {'problem': '有两个数学概念需要测试，第一个是判断一个整数是否是素数，第二个是验证一个数是否能被另一个数平方后且小于或等于这个数地整除。以下是数学问题的表述：\n\n1. 一个大于1的整数被称为素数，如果它没有正除数（除了1和它自身）。\n2. 一个整数X能被另一个数Y整除的条件是：Y的平方小于等于X，且X除以Y的余数为0。如果没有这样Y使得X能被整除，那么X是一个素数。\n\n根据以上定义，回答以下选择题：', 'questions': [{'question': '选择题 1：\n整数7是否是一个素数？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 2：\n整数2是否是一个素数？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 3：\n整数6是否能被整数3平方后且小于或等于这个数地整除？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 4：\n整数4是否能被整数3平方后且小于或等于这个数地整除？', 'options': ['是', '否'], 'answer': 'B'}], 'id': 'round1_test_data_003'}, {'problem': '有一个数字列表，你需要判断是否可以从这个列表中选择三个不同的数字，使得它们的和等于特定的数值。\n\n假设你有下列数字列表 `[1, 3, 5, 7]`，根据列表：', 'questions': [{'question': '选择题 1：\n使用列表中的三个不同数字，哪个数字是你可以形成的和为 11 的组合？', 'options': ['1 + 3 + 7', '3 + 5 + 7', '1 + 5 +

In [14]:
sorted_data

[{'problem': '有一个列表，找出该列表的最后一个元素。\n\n下列选项中哪个是列表 `[a, b, c, d]` 的最后一个元素？',
  'questions': [{'question': '选择题 1：',
    'options': ['a', 'b', 'c', 'd'],
    'answer': '不'}],
  'id': 'round1_test_data_002'},
 {'problem': '有两个数学概念需要测试，第一个是判断一个整数是否是素数，第二个是验证一个数是否能被另一个数平方后且小于或等于这个数地整除。以下是数学问题的表述：\n\n1. 一个大于1的整数被称为素数，如果它没有正除数（除了1和它自身）。\n2. 一个整数X能被另一个数Y整除的条件是：Y的平方小于等于X，且X除以Y的余数为0。如果没有这样Y使得X能被整除，那么X是一个素数。\n\n根据以上定义，回答以下选择题：',
  'questions': [{'question': '选择题 1：\n整数7是否是一个素数？',
    'options': ['是', '否'],
    'answer': 'A'},
   {'question': '选择题 2：\n整数2是否是一个素数？', 'options': ['是', '否'], 'answer': 'A'},
   {'question': '选择题 3：\n整数6是否能被整数3平方后且小于或等于这个数地整除？',
    'options': ['是', '否'],
    'answer': 'A'},
   {'question': '选择题 4：\n整数4是否能被整数3平方后且小于或等于这个数地整除？',
    'options': ['是', '否'],
    'answer': 'B'}],
  'id': 'round1_test_data_003'},
 {'problem': '有一个数字列表，你需要判断是否可以从这个列表中选择三个不同的数字，使得它们的和等于特定的数值。\n\n假设你有下列数字列表 `[1, 3, 5, 7]`，根据列表：',
  'questions': [{'question': '选择题 1：\n使用列表中的三个不同数字，哪个数字是你可以形成的和为 11 

In [15]:
# 从一组字典列表中提取特定格式的ID，然后找出在0到499范围内缺失的ID。
# 1.定义函数 find_missing_ids，接受一个参数 dict_list，表示包含多个字典的列表。
def find_missing_ids(dict_list):

    # 2.提取所有序号
    # 使用集合推导式，从每个字典的 id 键中提取最后三位，并转换为整数，形成一个集合 extracted_ids。
    # d['id'][-3:] 获取 id 的最后三位。
    # int(d['id'][-3:]) 将字符串转换为整数。
    extracted_ids = {int(d['id'][-3:]) for d in dict_list}
    
    # 3.创建0-500的序号集合
    # 使用 range(500) 生成0到499的整数序列，并转换为集合 all_ids。
    # set(range(500)) 创建一个包含0到499所有整数的集合。
    all_ids = set(range(500))
    
    # 4.找出缺失的序号
    # 使用集合差集运算 all_ids - extracted_ids 找出在 all_ids 中但不在 extracted_ids 中的元素，形成集合 missing_ids。
    missing_ids = all_ids - extracted_ids
    
    # 5.使用 sorted 对缺失的序号进行排序，并返回排序后的列表。
    return sorted(missing_ids)

# 示例字典列表
dict_list = sorted_data

# 找出缺失的序号
missing_ids = find_missing_ids(dict_list)
print("缺失的序号:", missing_ids)

缺失的序号: [0, 1, 4, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21, 23, 26, 28, 29, 30, 33, 34, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 50, 52, 53, 54, 55, 56, 59, 60, 62, 63, 65, 67, 68, 70, 71, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 90, 93, 94, 95, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 111, 113, 116, 117, 118, 119, 120, 122, 123, 124, 125, 127, 128, 130, 131, 132, 133, 134, 136, 139, 140, 141, 143, 144, 146, 147, 149, 150, 151, 153, 154, 160, 164, 165, 166, 170, 174, 175, 176, 177, 178, 180, 184, 185, 188, 189, 190, 192, 193, 195, 196, 197, 198, 199, 200, 203, 205, 206, 207, 208, 209, 210, 212, 214, 216, 218, 219, 224, 225, 226, 228, 229, 230, 231, 234, 236, 240, 242, 243, 244, 247, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 265, 267, 268, 269, 272, 274, 277, 278, 280, 281, 282, 287, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 306, 307, 308, 309, 310, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322

In [16]:
len(missing_ids)

326

In [17]:
# 读取 round1_test_data.jsonl 文件中的数据，检查其中是否有缺失的ID，并对这些缺失ID对应的条目进行处理和更新。
# 最后，对更新后的数据进行排序。

# 1.初始化一个空列表 data，这个列表在后续代码中未被使用。可能是多余的或用于其他部分代码中。
data  = []

# 2.打开文件读取数据
# 使用 with open('round1_test_data.jsonl') as reader 以只读模式打开文件 round1_test_data.jsonl，并将文件对象赋值给 reader。
# 使用 enumerate(reader) 对文件逐行读取，并为每一行分配一个索引 id。
with open('round1_test_data.jsonl') as reader:
    for id,line in enumerate(reader):

        # 3.检查和处理缺失的ID
        # if id in missing_ids：检查当前行的索引 id 是否在 missing_ids 集合中。
        # sample = json.loads(line)：将当前行的JSON格式字符串解析为Python对象 sample。
        # for question in sample['questions']：遍历 sample 中的所有问题 questions。
        # question['answer'] = 'A'：将每个问题的 answer 键值设置为 'A'，这可能是为了填补缺失的答案。
        # sorted_data.append(sample)：将处理后的 sample 添加到 sorted_data 列表中。
        if(id in missing_ids):
            sample = json.loads(line)
            for question in sample['questions']:
                question['answer'] = 'A'
            sorted_data.append(sample)

# 4.排序数据
# 使用 sorted 函数对 sorted_data 列表进行排序，排序键为每个数据项 id 的最后三位数字。
# key=lambda x: int(str(x['id'])[-3:])：
# str(x['id'])：将 id 转换为字符串。
# str(x['id'])[-3:]：获取 id 字符串的最后三位。
# int(str(x['id'])[-3:])：将最后三位字符串转换为整数，作为排序键。
sorted_data = sorted(sorted_data, key=lambda x: int(str(x['id'])[-3:]))

In [18]:
# 将 sorted_data 中的数据写入到 upload.jsonl 文件中
# 1.打开文件以写入format
# 使用 with open('upload.jsonl', 'w') as writer 以写入模式 ('w') 打开或创建文件 upload.jsonl。
# writer 是文件对象，提供写入方法。
# with 语句确保文件在写入操作完成后自动关闭，无论是否发生异常。
with open('upload.jsonl', 'w') as writer:

    # 2.遍历 sorted_data 列表中的每个元素 sample。
    # sorted_data 包含处理后的数据，每个元素通常是一个字典，表示一个问题及其相关信息。
    for sample in sorted_data:

        # 3.使用 json.dumps(sample, ensure_ascii=False) 将 sample 转换为JSON格式的字符串。
        # sample 是一个字典。
        # ensure_ascii=False 确保非ASCII字符（如中文）能够正确显示，而不是转义为 \uXXXX 格式。
        # writer.write(json.dumps(sample, ensure_ascii=False)) 将转换后的JSON字符串写入文件。
        writer.write(json.dumps(sample, ensure_ascii=False))

        # 4.写入换行符
        # 写入一个换行符 '\n'，确保每个JSON对象占用文件中的一行。
        # 这使得文件 upload.jsonl 是一个符合JSON Lines格式的文件，每行是一个独立的JSON对象。
        writer.write('\n')

In [19]:
print("!!!!!! baseline1 运行成功！快去左侧下载 upload.jsonl ！ 去赛事官网完成提交吧！")

!!!!!! baseline1 运行成功！快去左侧下载 upload.jsonl ！ 去赛事官网完成提交吧！
